In [1]:
import os
import sys
import re

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sys.path.insert(0, '../src')
from features import *
from utils import *

In [3]:
sns.set(rc={'figure.figsize':(20,5)})

In [4]:
mbit_rate = 1/125000

low_fp = '../data/240p/' 
threesixty_fp = '../data/360p/' 
med_fp = '../data/480p/'
seventwenty_fp = '../data/720p/' 
high_fp = '../data/1080p/' 

In [5]:
low_dfs = []
for file in os.listdir(low_fp):
    if file != '.ipynb_checkpoints':
        low_dfs.append(pd.read_csv(low_fp+file))
    
threesixty_dfs = []
for file in os.listdir(threesixty_fp):
    if file != '.ipynb_checkpoints':
        threesixty_dfs.append(pd.read_csv(threesixty_fp+file))
        
med_dfs = []
for file in os.listdir(med_fp):
    if file != '.ipynb_checkpoints':
        med_dfs.append(pd.read_csv(med_fp+file))
        
seventwenty_dfs = []
for file in os.listdir(seventwenty_fp):
    if file != '.ipynb_checkpoints':
        seventwenty_dfs.append(pd.read_csv(seventwenty_fp+file))
    
high_dfs = []
for file in os.listdir(high_fp):
    if file != '.ipynb_checkpoints':
        high_dfs.append(pd.read_csv(high_fp+file))

In [6]:
# stdoan_low = pd.read_csv(low_fp + 'stdoan-101-action-240p-20201127.csv')
# stdoan_med = pd.read_csv(med_fp + 'stdoan-101-action-480p-20201127.csv')
# stdoan_high = pd.read_csv(high_fp + 'stdoan-101-action-1080p-20201127.csv')

In [6]:
low_ms = []
for df in low_dfs:
    low_ms.append(convert_ms_df(df,True))
    
threesixty_ms = []
for df in threesixty_dfs:
    threesixty_ms.append(convert_ms_df(df,True))
    
med_ms = []
for df in med_dfs:
    med_ms.append(convert_ms_df(df,True))
    
seventwenty_ms = []
for df in seventwenty_dfs:
    seventwenty_ms.append(convert_ms_df(df,True))
    
high_ms = []
for df in high_dfs:
    high_ms.append(convert_ms_df(df,True))

In [8]:
# low_ms = convert_ms_df(stdoan_low, True)
# med_ms = convert_ms_df(stdoan_med, True)
# high_ms = convert_ms_df(stdoan_high, True)

In [7]:
low_resamples = []
for df in low_ms:
    low_resamples.append(df.resample('500ms', on='Time').sum())
    
threesixty_resamples = []
for df in threesixty_ms:
    threesixty_resamples.append(df.resample('500ms', on='Time').sum())
    
med_resamples = []
for df in med_ms:
    med_resamples.append(df.resample('500ms', on='Time').sum())
    
seventwenty_resamples = []
for df in seventwenty_ms:
    seventwenty_resamples.append(df.resample('500ms', on='Time').sum())
    
high_resamples = []
for df in high_ms:
    high_resamples.append(df.resample('500ms', on='Time').sum())

In [8]:
# low_resample = low_ms.resample('500ms', on='Time').sum()
# med_resample = med_ms.resample('500ms', on='Time').sum()
# high_resample = high_ms.resample('500ms', on='Time').sum()


## Aggregate Features

In [9]:
## take the aggregate features of the whole chunk; download and upload
def agg_feat(df, col):
    return [np.mean(df[col]), np.std(df[col])]

## take the ratio of upload:download packets
def pkt_ratio(df):
    ms_df = convert_ms_df(df, True)
    local = np.sum(ms_df['pkt_src'] == '1') 
    server = np.sum(ms_df['pkt_src'] == '2') 
    return local / server

## take the ratio of upload:download bytes
def bytes_ratio(df):
    local = df['1->2Bytes'].sum()
    server = df['2->1Bytes'].sum()
    return local / server

## Peak Related Aggregate Features

In [226]:
## finds the peaks with mean + 2(1) std
## run the above aggregate functions on the peaks only??

def get_peak_loc(df, col, invert=False):
  'invert arg allows you to get values not considered peaks'
  df_avg = df[col].mean()
  df_std = df[col].std()
  
  threshold = df_avg + (1 * df_std)
  
  if invert:
    return np.array(df[col] < threshold)
  
  else:
    return np.array(df[col] > threshold)

## np.mean, np.var, np.std - think of more?  
def peak_time_diff(df, col):
  '''
  mess around with the different inputs for function. 
  variance seems to inflate the difference betweent the two the most with litte
  to no data manipulation. however, currently trying things like
  squaring the data before taking the aggregate function to exaggerate
  differences (moderate success??)
  '''
  peaks = df[get_peak_loc(df, col)]
  peaks['Time'] = peaks['Time'] - peaks['Time'].min()
  time_diff = np.diff(peaks['Time'] ** 2)
  return [np.mean(time_diff), np.std(time_diff)]

In [11]:
from scipy.signal import find_peaks

def peak_times(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    if list(peaks) == []:
        return [-1]
    times = df.iloc[peaks]['Time'].values
    time_between_peaks = [times[i]-times[i-1]for i in range(1,len(times))]
    #print(time_between_peaks)
    #time_between_peaks[0]=0
    if time_between_peaks == []:
        return -1
    return time_between_peaks

def num_peaks(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    return len(peaks)

## Spectral Features

In [12]:
def spectral_features(df, col):

    """
    welch implemention of spectral features
    resample the data before inputting (might change prereq depending on
    resource allocation)
    """

    f, Pxx_den = sp.signal.welch(df[col], fs=2)
    Pxx_den = np.sqrt(Pxx_den)

    peaks = sp.signal.find_peaks(Pxx_den)[0]
    prominences = sp.signal.peak_prominences(Pxx_den, peaks)[0]

    idx_max = prominences.argmax()
    loc_max = peaks[idx_max]

    return [f[loc_max], Pxx_den[loc_max], prominences[idx_max]]

## Chunking & Feature creation

In [13]:
## wip; need to decide chunk size eventually
## should we also make this chunking feature be our feature creation?

def chunk_data(df, interval=60):

    """
    takes in a filepath to the data you want to chunk and feature engineer
    chunks our data into a specified time interval
    each chunk is then turned into an observation to be fed into our classifier
    """

    df_list = []
    
    df['Time'] = df['Time'] - df['Time'].min()
    
    total_chunks = np.floor(df['Time'].max() / interval).astype(int)

    for chunk in np.arange(total_chunks):
      
        start = chunk * interval
        end = (chunk+1) * interval

        temp_df = (df[(df['Time'] >= start) & (df['Time'] < end)])
        
        df_list.append(temp_df)
        
    return df_list

In [227]:
def create_features(dfs, interval=60):

  features = [
    'dwl_peak_freq',
    'dwl_peak_prom',
    'dwl_max_psd',
    'dwl_bytes_avg',
    'dwl_bytes_std',
    'dwl_peak_avg',
    'dwl_peak_std',
    'upl_peak_freq',
    'upl_peak_prom',
    'upl_max_psd',
    'upl_bytes_avg',
    'upl_bytes_std',
    'upl_peak_avg',
    'upl_peak_std',
    'dwl_time_peak',#'IMAN_up_time_peak',
      'dwl_num_peak'#,'IMAN_up_num_peak'
  ]  

  vals = []
  for df in dfs:
      df_chunks = chunk_data(df, interval)

      for chunk in df_chunks:

        preproc = convert_ms_df(chunk, True)
        upl_bytes = preproc[preproc['pkt_src'] == '1'].resample('500ms', on='Time').sum()
        dwl_bytes = preproc[preproc['pkt_src'] == '2'].resample('500ms', on='Time').sum()

        ## spectral features
        dwl_spectral = spectral_features(dwl_bytes, 'pkt_size')
        upl_spectral = spectral_features(upl_bytes, 'pkt_size')

        ## aggregate features
        dwl_agg = agg_feat(chunk, '2->1Bytes')
        upl_agg = agg_feat(chunk, '1->2Bytes')

        ## peak features
        dwl_peak = peak_time_diff(chunk, '2->1Bytes')
        upl_peak = peak_time_diff(chunk, '1->2Bytes')
        
        ## iman's time between peak 
        iman_dwn_time_peak = np.mean(peak_times(chunk,'2->1Bytes',1000000))
        #iman_up_time_peak = np.mean(peak_times(chunk,'1->2Bytes',50000))
        
        ## iman's num peak
        iman_dwn_num_peak = num_peaks(chunk,'2->1Bytes',1000000)
        #iman_up_num_peak = num_peaks(chunk,'1->2Bytes',50000)


        
        feat_val = np.hstack((
          dwl_spectral,
          dwl_agg,
          dwl_peak,
          upl_spectral,
          upl_agg,
          upl_peak,
            iman_dwn_time_peak,#iman_up_time_peak,
            iman_dwn_num_peak,#iman_up_num_peak
        ))

        vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

## Dev Playground

In [228]:
%%time
low_feat = create_features(low_dfs, 100)
threesixty_feat = create_features(threesixty_dfs, 100)
med_feat = create_features(med_dfs, 100)
seventwenty_feat = create_features(seventwenty_dfs, 100)
high_feat = create_features(high_dfs, 100)

CPU times: user 28.6 s, sys: 468 ms, total: 29.1 s
Wall time: 29.3 s


In [261]:
low_feat['resolution'] = '240p'
threesixty_feat['resolution'] = '360p'
med_feat['resolution'] = '480p'
seventwenty_feat['resolution'] = '720p'
high_feat['resolution'] = '1080p'

In [262]:
training = pd.concat([low_feat, threesixty_feat, med_feat,seventwenty_feat, high_feat]).reset_index(drop=True)

In [263]:
## SELECT SUBSETS OF FEATURES

#training = training[['dwl_bytes_avg','dwl_peak_prom','upl_bytes_std','dwl_bytes_std','upl_peak_std','resolution']]
#training = training[['dwl_bytes_avg','upl_max_psd','dwl_max_psd','upl_peak_prom','dwl_num_peak','dwl_peak_prom','resolution']]
#training = training[['dwl_max_psd','upl_max_psd','dwl_peak_prom','upl_peak_prom','dwl_num_peak','dwl_bytes_avg','upl_bytes_std','upl_bytes_avg','resolution']]


In [264]:
training

,dwl_peak_freq,dwl_peak_prom,dwl_max_psd,dwl_bytes_avg,dwl_bytes_std,dwl_peak_avg,dwl_peak_std,upl_peak_freq,upl_peak_prom,upl_max_psd,upl_bytes_avg,upl_bytes_std,upl_peak_avg,upl_peak_std,dwl_time_peak,dwl_num_peak,resolution
0,0.030303,3.498658e+05,3.441002e+05,72271.795699,256539.110163,740.571429,1057.401783,0.282828,12923.219670,12150.389735,3766.440860,10379.952163,800.000000,1001.528082,36.000000,2.0,240p
1,0.130000,2.838183e+05,2.501136e+05,83451.321839,316051.124807,1380.166667,963.684322,0.130000,12377.000916,10827.771453,4115.344828,12564.890968,1380.166667,963.684322,36.500000,3.0,240p
2,0.030151,3.503755e+05,3.430110e+05,74871.558140,271623.742644,980.000000,1038.429006,0.060000,13348.023387,12677.011516,3696.953488,10854.020391,980.000000,1038.429006,35.000000,3.0,240p
3,0.336735,3.230138e+05,3.209500e+05,88509.135135,284640.334331,1040.166667,901.514728,0.061224,13571.342361,13239.819300,4347.527027,11384.471692,1040.166667,901.514728,34.500000,3.0,240p
4,0.030769,3.487811e+05,3.236499e+05,49942.650485,191380.472816,620.166667,893.730462,0.196891,16734.177901,14314.387074,9844.368932,50788.781600,1.000000,0.000000,-1.000000,1.0,240p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,0.100503,1.097268e+06,1.043545e+06,253738.413043,580189.681359,902.500000,585.807349,0.100000,53262.247600,52123.889925,13293.500000,27622.507815,902.500000,585.807349,10.500000,9.0,1080p
219,0.092308,9.765048e+05,8.906232e+05,217527.819444,516862.726467,882.000000,1309.223625,0.092308,46129.569359,41808.893237,11329.333333,24669.122888,882.000000,1309.223625,10.400000,6.0,1080p
220,0.183673,5.251847e+05,4.571161e+05,229211.126437,575108.238130,921.600000,904.036526,0.183673,25874.543373,24163.202914,11889.528736,27504.462558,921.600000,904.036526,13.333333,7.0,1080p
221,0.438776,8.523427e+05,8.051908e+05,221431.108108,578239.976452,435.125000,414.244927,0.122449,40051.038123,37570.814535,11485.864865,27438.619912,435.125000,414.244927,8.428571,8.0,1080p


In [265]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score

In [266]:
X, y = training.drop(columns=['resolution']), training['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 4)

In [273]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
#classifier = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier.fit(X_train, y_train)
#classifier.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [274]:
## RUN ON TEST DATA

#low_test = pd.read_csv('../data/test/sgs008-109-action-1080p-20210202.csv')
#low_test = pd.read_csv('../data/test/sgs008-109-action-720p-20210213.csv')
#low_test = pd.read_csv('../data/test/stdoan-102-action-720p-20201206.csv')


low_feat = create_features([low_test], 100)

test_low = low_feat#[['dwl_max_psd','upl_max_psd','dwl_peak_prom','upl_peak_prom','dwl_num_peak','dwl_bytes_avg','upl_bytes_std','upl_bytes_avg']]
y_pred = classifier.predict(test_low)
y_pred

array(['1080p', '720p', '720p', '1080p', '720p', '720p', '1080p', '1080p',
       '1080p', '1080p', '1080p', '720p', '720p', '1080p', '720p', '720p',
       '1080p', '720p'], dtype=object)

In [275]:
## FOR VALIDATION SET

y_pred = classifier.predict(X_test)

In [276]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Group'], colnames=['Predicted Group']))

Predicted Group,1080p,240p,360p,480p,720p
Actual Group,,,,,
1080p,11,0,0,1,0
240p,0,4,1,0,0
360p,0,2,7,1,1
480p,0,0,1,16,1
720p,2,0,0,0,8


In [277]:
f1_score(y_test, y_pred, average=None)

array([0.88      , 0.72727273, 0.7       , 0.88888889, 0.8       ])

In [278]:
accuracy_score(y_test, y_pred)

0.8214285714285714

In [279]:
# features = ['dwl_bytes_avg','dwl_peak_prom','upl_bytes_std','dwl_bytes_std','upl_peak_std']
# importances = classifier.feature_importances_
# indices = np.argsort(importances)[::-1]
# for i in indices:
#     print(features[i],': ',importances[i])
    

In [280]:
# features = ['dwl_bytes_avg','upl_max_psd','dwl_max_psd','upl_peak_prom','dwl_num_peak','dwl_peak_prom']
# importances = classifier.feature_importances_
# indices = np.argsort(importances)[::-1]
# for i in indices:
#     print(features[i],': ',importances[i])

In [281]:
features = ['dwl_peak_freq','dwl_peak_prom','dwl_max_psd','dwl_bytes_avg','dwl_bytes_std','dwl_peak_avg',
            'dwl_peak_std','upl_peak_freq','upl_peak_prom','upl_max_psd','upl_bytes_avg','upl_bytes_std',
            'upl_peak_avg','upl_peak_std','dwl_time_peak','dwl_num_peak']
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(features[i],': ',importances[i])
    

upl_max_psd :  0.11778561076842109
dwl_bytes_avg :  0.10960748105793466
upl_peak_prom :  0.10182601757952935
dwl_peak_prom :  0.09213709181112298
dwl_bytes_std :  0.08159211937725258
dwl_max_psd :  0.07261900114728903
upl_bytes_avg :  0.07162313477015164
upl_bytes_std :  0.06717286348384563
dwl_num_peak :  0.04902615285964612
dwl_time_peak :  0.04877282060674773
upl_peak_avg :  0.0472601473539359
upl_peak_std :  0.035245518484478794
dwl_peak_std :  0.03293344829082329
dwl_peak_freq :  0.02739914519803026
dwl_peak_avg :  0.026685600206685797
upl_peak_freq :  0.018313847004105134


In [32]:
# ## testing that feature method functions correctly

# l_start = 0 
# l_end = 60

# test_chunk = stdoan_low.copy()
# test_chunk['Time'] = test_chunk['Time'] - test_chunk['Time'].min()
# low_chunk = stdoan_low[(stdoan_low['Time'] >= 0) & (stdoan_low['Time'] < 60)]

# low_chunk_ms = convert_ms_df(low_chunk, True)

# upl_ms = low_chunk_ms[low_chunk_ms['pkt_src'] == '1']
# dwl_ms = low_chunk_ms[low_chunk_ms['pkt_src'] == '2']

# dwl_chunk_rs = dwl_ms.resample('500ms', on='Time').sum()

# f_dwl, Pxx_dwl = sp.signal.welch(dwl_chunk_rs['pkt_size'], fs=2)